In [629]:
# loading package
import numpy as np
import pandas as pd
import copy
import igraph
from scipy.optimize import minimize
from scipy.special import digamma
import networkx as nx

## Loading data

Set the initial parameters for $\alpha, \beta, \delta, \gamma, \lambda$

In [630]:
np.random.seed(1)

p = 5
E_true  = np.zeros((p, p), dtype=int)

for j in range(1, p):
    E_true[j, j - 1] = 1

alpha_true = np.zeros((p, p))
alpha_true[E_true == 1] = np.random.uniform(0.5, 2, size=np.sum(E_true))
beta_true = np.zeros((p, p))
beta_true[E_true == 1] = np.random.uniform(-2, -0.5, size=np.sum(E_true))
delta_true = np.random.uniform(-1.5, -1, size=p)
gamma_true = np.random.uniform(1, 1.5, size=p)
lambda_true = np.exp(np.random.uniform(-2, 2, size=p))


# Generate synthetic data from the specified linear ZiG-DAG
n = 50  # Sample size
dat = np.zeros((n, p))
#order_nodes = list(igraph.topological_sort(igraph.Graph.Erdos_Renyi(n=p, p=0.5, directed=True)))
order_nodes = np.arange(0,5)

for j in order_nodes:
    pi_true = np.exp(np.dot(dat, alpha_true[j, :]) + delta_true[j])
    pi_true = pi_true / (1 + pi_true)
    pi_true[np.isnan(pi_true)] = 1
    mu_true = np.exp(np.dot(dat, beta_true[j, :]) + gamma_true[j])
    # use r output at first for following data
    #dat[:, j] = (1 - np.random.binomial(1, pi_true, size=n)) * rhP(n, lambda_true[j], mu_true)

dat = pd.read_csv('syn_data.csv',sep=',')
dat = np.array(dat)


**Linear ZiGDAG**

In [563]:
def f11_cpp(p, a, b, Iter, tol):
    
    c = np.copy(a)
    d = np.copy(b)
    
    fac = 1
    temp = fac
    series = 0
    
    for e in range(Iter):
        fac *= (c/d) * (p/(e+1))

        if np.isnan(fac): fac = 0
        series = temp + fac
        
        if (not np.isfinite(series) or np.abs(series - temp) < tol): 
            return series
        
        temp = series
        
        c += 1
        d += 1
    return series

def f11_for_each(z, a, b, Iter, tol):
    
    '''
    
    z: vector Mu_vector is 2D vector
    x: 1
    y: lambda
    
    '''
    
    num = len(z)
    val = np.zeros((num, 1))
    
    for i in range(num):
        val[i, 0] = f11_cpp(z[i, 0], a, b, Iter, tol)
    
    return np.array(val)

def log_ascfacto(z, n):
    p = len(n)
    out = np.zeros((p, 1))
    
    for i in range(p):
        for j in range(n[i]):
            out[i] += np.log(z+j)
            
    return out
    
    
def dZIHP_cpp(z, y, x, lower, upper):

    z = z.reshape(1,-1)

    pn, pp = x.shape
    x1 = np.concatenate([x, np.ones((n,1))], axis = 1)

    Pi = np.exp(x1 @ z[:,0:pp+1].T)
    Pi = Pi/(1+Pi)
    Pi[~np.isfinite(Pi)] = 1
    
    Mu = np.exp(x1 @ z[:,pp+1:2*pp+2].T)
    Lambda = np.copy(np.exp(z[:,-1]))
    
    Y0 = np.where(y<=1e-8)
    Y1 = np.where(y>1e-8)
    eval_F11 = f11_for_each(Mu, 1, Lambda, 10000, 1e-8)
    
    #return Y0, Y1, eval_F11, Pi, Mu
    # keep working on likelihood function
    llik = np.sum(np.log(Pi[Y0] + (1-Pi[Y0])/eval_F11[Y0]))+np.sum(np.log(1-Pi[Y1])-log_ascfacto(Lambda, y[Y1])-np.log(eval_F11[Y1]) + (y[Y1].reshape(-1,1) % np.log(Mu[Y1])))

    if ((not np.isfinite(llik)) and (llik<0)):
        return lower
    elif ((not np.isfinite(llik)) and (llik>0)):
        return upper
    else:
        return llik

def gradF11b_cpp(z, b, Iter, tol):
    
    c = np.copy(b)
    
    con = digamma(c) * f11_cpp(z, 1, c, Iter, tol)
    fac = digamma(c)
    temp = fac
    series = 0
    
    for i in range(Iter):
        fac *= (digamma(c+1)/digamma(c)) * (z/c)
        
        if np.isnan(fac): fac = 0
        series = temp + fac
        
        if (not np.isfinite(series)) or (np.abs(series - temp) < tol):
            return (con - series)
        
        temp = series
        c += 1
        
    return (con-series)

def gradF11b_for_each(z, b, Iter, tol):
    
    '''
    
    z: vector Mu_vector is 2D vector
    x: 1
    y: lambda
    
    '''
    
    num = len(z)
    val = np.empty((num, 1))

    for i in range(num):
        val[i, 0] = gradF11b_cpp(z[i, 0], b, Iter, tol)
        
    return val


def gradZIHP_cpp(z, y, x):
    
    z = z.reshape(1,-1)

    pn, pp = x.shape
    x1 = np.concatenate([x, np.ones((n,1))], axis = 1)

    Pi = np.exp(x1 @ z[:,0:pp+1].T)
    Pi = Pi/(1+Pi)
    Pi[~np.isfinite(Pi)] = 1

    Mu = np.exp(x1 @ z[:,pp+1:2*pp+2].T)
    Lambda = np.exp(z[:,-1])
    
    #print('Lambda1: ', Lambda)
    Y0 = np.where(y<=1e-8)
    Y1 = np.where(y>1e-8)
    X10 = x1[Y0]
    X11 = x1[Y1]
    
    eval1_F11 = f11_for_each(Mu, 1, Lambda, 10000, 1e-8)
    eval2_F11 = f11_for_each(Mu, 2, Lambda+1, 10000, 1e-8)
    eval_gradF11b = gradF11b_for_each(Mu, Lambda, 10000, 1e-8)
    
    #print('Lambda2: ', Lambda)
    dZIHP0 = Pi[Y0] + (1-Pi[Y0])/eval1_F11[Y0]
    
    grad = np.zeros(2 * pp + 3)
    # C++ code for p1
    #arma::sum(X10.each_col() % ((1 - Pi.elem(Y0)) % (Pi.elem(Y0) - Pi.elem(Y0) / eval1_F11.elem(Y0)) / dZIHP0), 0) - arma::sum(X11.each_col() % Pi.elem(Y1), 0);
    p1 = np.sum(X10 % ((1 - Pi[Y0]) % ((Pi[Y0] - (Pi[Y0]/ eval1_F11[Y0])) / dZIHP0)), axis = 0) - np.sum(X11 % Pi[Y1], axis =0) 
    
    # C++ code for p2
    # -arma::sum(X10.each_col() % (Mu.elem(Y0) % (1 - Pi.elem(Y0)) / Lambda % eval2_F11.elem(Y0) / arma::square(eval1_F11.elem(Y0)) / dZIHP0), 0) -
    # arma::sum(X11.each_col() % (Mu.elem(Y1) / Lambda % eval2_F11.elem(Y1) / eval1_F11.elem(Y1) - y.elem(Y1)), 0);
    p2 = -np.sum(X10 % Mu[Y0] % ((1-Pi[Y0])/Lambda) % (eval2_F11[Y0]/np.square(eval1_F11[Y0])/dZIHP0), axis = 0) -np.sum(X11 % (((Mu[Y1]/Lambda) % (eval2_F11[Y1]/eval1_F11[Y1])) -y[Y1].reshape(-1,1)), axis = 0)

    # C++ code for p3
    # -arma::accu((1 - Pi.elem(Y0)) % eval_gradF11b.elem(Y0) / arma::square(eval1_F11.elem(Y0)) / dZIHP0) - 
    # arma::accu(eval_gradF11b.elem(Y1) / eval1_F11.elem(Y1) + digamma_arma(Lambda + y.elem(Y1)) - R::digamma(Lambda))) * Lambda;
    p3 = (-np.sum((1-Pi[Y0]) % (eval_gradF11b[Y0]/np.square(eval1_F11[Y0])/dZIHP0), axis = 0) -np.sum((eval_gradF11b[Y1]/eval1_F11[Y1] + digamma(Lambda + y[Y1].reshape(-1,1)) - digamma(Lambda)), axis = 0)) * Lambda
    
    grad[0:pp+1] = p1
    grad[pp+1: 2*pp+2] = p2
    grad[2*pp+2] = p3

    return grad


def create_dag(M):
    G = nx.DiGraph()
    snode = np.where(M == 1)[0]
    enode = np.where(M == 1)[1]
    
    for i in range(len(snode)):
        G.add_edge(snode[i], enode[i])
    
    return G

def initialpars(ps):
    start_pars = []
    
    for e in ps:
        if type(e) == np.ndarray:
            start_pars.extend(e)
        elif type(e) == np.float64:
            start_pars.append(e)

    return (np.array(start_pars))

def acc(trueE, predE, Type):
    #tp/(tp+fn)
    #fp/(tp+fp)
    
    tp = 0
    fn = 0
    fp = 0
    
    dx, dy = trueE.shape
    for i in range(dx):
        for j in range(dy):
            if trueE[i, j] == 1 and predE[i, j] == 1:
                tp += 1
            elif trueE[i, j] == 1 and predE[i, j] == 0:
                fn += 1
            elif trueE[i, j] == 0 and predE[i, j] == 1:
                fp += 1
    
    if Type == 'TPR':
        return tp/(tp+fn)
    
    elif Type == 'FDR':
        return fp/(tp+fp)
    

In [558]:
def hc_linear_zihp(dat, starting_dag = None, maxiter=500, tol=1e-12, optim_control=None, verbose=False):
    
    n, p = dat.shape

    #if optim_control is None:
        #optim_control = {"fnscale": -1, "maxiter": 10000, "reltol": 1.0e-8}

    if starting_dag is None:
        starting_dag = np.zeros((p, p))
        
    bic_curr = np.empty(p)
    est_curr = {
        "E": starting_dag,
        #"alpha": np.ones((p, p))*0.5,
        "alpha": np.ones((p, p))*0,
        #"beta": np.ones((p, p))*0.5,
        "beta": np.ones((p, p))*0,
        "delta": np.zeros(p),
        #"gamma": np.ones(p)*0.6,
        "gamma": np.ones(p)*0,
        "lambda": np.ones(p),
    }

    for j in range(p):

        pa_j = (est_curr['E'][j, :] == 1)

        if np.array(est_curr["alpha"][j, pa_j]).size == 0 and np.array(est_curr["beta"][j, pa_j]).size == 0:
            start_j = [est_curr["delta"][j], est_curr["gamma"][j],
                        np.log(est_curr["lambda"][j])]
        else:
            start_j = [(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                        np.log(est_curr["lambda"][j])]

        start_j = initialpars(start_j)
        pars = np.array(start_j)

        out_j = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, j], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)
         , x0=pars, jac=lambda z: gradZIHP_cpp(z, dat[:, j], dat[:, pa_j]), method='BFGS')

        p_j = sum(pa_j)
        bic_curr[j] = -2 * out_j.fun + (2 * p_j + 3) * np.log(n)

        if p_j > 0:
            est_curr["alpha"][j, pa_j] = out_j.x[0 : p_j]
            est_curr["beta"][j, pa_j]  = out_j.x[(p_j + 1) : (2 * p_j + 1)]

        est_curr["delta"][j]  = out_j.x[p_j]
        est_curr["gamma"][j]  = out_j.x[2 * p_j + 1]
        est_curr["lambda"][j] = np.exp(out_j.x[2 * p_j + 2])
    
    bic_iter = bic_curr.copy()
    est_iter = est_curr.copy()
    
    # start hill climbing
    
    bic_iter = bic_curr.copy()
    est_iter = est_curr.copy()

    maxiter = 500

    for Iter in range(maxiter):
        IMPROV = False
        for j in range(p):
            for k in range(p):

                if j == k:
                    continue

                if est_curr["E"][j, k] == 0:

                    # add edge
                    E_cand = est_curr["E"].copy()
                    E_cand[j, k] = 1

                    # create dag for E_cand
                    G_cand = create_dag(E_cand)
                    if not nx.is_directed_acyclic_graph(G_cand): 
                        continue

                    print("Add Edge from {0} to {1}".format(j, k))

                    pa_j = (E_cand[j, ] == 1)
                    if np.array(est_curr["alpha"][j, pa_j]).size == 0 and np.array(est_curr["beta"][j, pa_j]).size == 0:
                        start_j = [est_curr["delta"][j], est_curr["gamma"][j],
                                    np.log(est_curr["lambda"][j])]
                    else:
                        start_j = [(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                                    np.log(est_curr["lambda"][j])]

                    start_j = initialpars(start_j)
                    pars = np.array(start_j)


                    out_j = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, j], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)
                         , x0=pars, jac=lambda z: gradZIHP_cpp(z, dat[:, j], dat[:, pa_j]), method='BFGS')
                    #out_j = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, j], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)
                         #, x0=pars, method='BFGS')

                    p_j = sum(pa_j)

                    bic_cand = bic_curr.copy()
                    #2pj+3 -> 2pj
                    bic_cand[j] = -2 * out_j.fun + (2 * p_j + 3) * np.log(n)

                    print("cand bic: ",np.round(np.sum(bic_cand),4))
                    print("iter bic: ",np.round(np.sum(bic_iter),4))

                    if np.isfinite(np.sum(bic_cand)) and np.sum(bic_cand) < np.sum(bic_iter) - tol:
                        IMPROV = True
                        bic_iter = bic_cand
                        est_iter = est_curr.copy()
                        est_iter["E"] = E_cand.copy()
                        est_iter["alpha"][j, pa_j] = out_j.x[:p_j]
                        est_iter["beta"][j, pa_j] = out_j.x[(p_j + 1) : (2 * p_j + 1)]
                        est_iter["delta"][j] = out_j.x[p_j]
                        est_iter["gamma"][j] = out_j.x[2 * p_j + 1]
                        est_iter["lambda"][j] = np.exp(out_j.x[2 * p_j + 2])

                else:

                    # delete edge
                    E_cand = est_curr["E"].copy()
                    E_cand[j, k] = 0

                    print("Remove Edge from {0} to {1}".format(j, k))

                    pa_j = (E_cand[j, ] == 1)
                    if np.array(est_curr["alpha"][j, pa_j]).size == 0 and np.array(est_curr["beta"][j, pa_j]).size == 0:
                        start_j = [est_curr["delta"][j], est_curr["gamma"][j],
                                    np.log(est_curr["lambda"][j])]
                    else:
                        start_j = [(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                                    np.log(est_curr["lambda"][j])]

                    start_j = initialpars(start_j)
                    pars = np.array(start_j)

                    out_j = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, j], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)
                         , x0=pars, jac=lambda z: gradZIHP_cpp(z, dat[:, j], dat[:, pa_j]), method='BFGS')
                    #out_j = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, j], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)
                         #, x0=pars, method='BFGS')

                    p_j = sum(pa_j)
                    bic_cand = bic_curr.copy()
                    #2pj+3 -> 2pj
                    bic_cand[j] = -2 * out_j.fun + (2 * p_j + 3) * np.log(n)
                    print("cand bic: ",np.round(np.sum(bic_cand),4))
                    print("iter bic: ",np.round(np.sum(bic_iter),4))

                    if np.isfinite(np.sum(bic_cand)) and np.sum(bic_cand) < np.sum(bic_iter) - tol:

                        IMPROV = True
                        bic_iter = bic_cand
                        est_iter = est_curr.copy()
                        est_iter["E"] = E_cand.copy()
                        est_iter["alpha"][j, pa_j] = out_j.x[:p_j]
                        est_iter["beta"][j, pa_j] = out_j.x[(p_j + 1) : (2 * p_j + 1)]
                        est_iter["delta"][j] = out_j.x[p_j]
                        est_iter["gamma"][j] = out_j.x[2 * p_j + 1]
                        est_iter["lambda"][j] = np.exp(out_j.x[2 * p_j + 2])


        id_edge = np.argwhere(est_curr['E'] == 1)
        n_rev = id_edge.shape[0]

        if n_rev > 0:
            for l in range(n_rev):
                j = id_edge[l, 0]
                k = id_edge[l, 1]
                E_cand = est_curr['E'].copy()
                E_cand[j, k] = 0
                E_cand[k, j] = 1

                # create dag for E_cand
                G_cand = create_dag(E_cand)
                if not nx.is_directed_acyclic_graph(G_cand): 
                    continue

                pa_j = (E_cand[j, ] == 1)

                if np.array(est_curr["alpha"][j, pa_j]).size == 0 and np.array(est_curr["beta"][j, pa_j]).size == 0:
                    start_j = [est_curr["delta"][j], est_curr["gamma"][j],
                                np.log(est_curr["lambda"][j])]
                else:
                    start_j = [(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                                np.log(est_curr["lambda"][j])]

                start_j = initialpars(start_j)
                pars = np.array(start_j)

                out_j = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, j], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)
                         , x0=pars, jac=lambda z: gradZIHP_cpp(z, dat[:, j], dat[:, pa_j]), method='BFGS')
                #out_j = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, j], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)
                     #, x0=pars, method='BFGS')

                pa_k = (E_cand[k, ] == 1)
                if np.array(est_curr["alpha"][k, pa_k]).size == 0 and np.array(est_curr["beta"][k, pa_k]).size == 0:
                    start_k = [est_curr["delta"][k], est_curr["gamma"][k],
                                np.log(est_curr["lambda"][k])]
                else:
                    start_k = [(est_curr["alpha"][k, pa_k]), est_curr["delta"][k], est_curr["beta"][k, pa_k], est_curr["gamma"][k],
                                np.log(est_curr["lambda"][k])]

                start_k = initialpars(start_k)
                parsk = np.array(start_k)

                out_k = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, k], dat[:, pa_k], -np.finfo(float).max, np.finfo(float).max)
                     , x0=parsk, jac=lambda z: gradZIHP_cpp(z, dat[:, k], dat[:, pa_k]), method='BFGS')
                #out_k = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, k], dat[:, pa_k], -np.finfo(float).max, np.finfo(float).max)
                         #, x0=parsk, method='BFGS')

                p_j = np.sum(pa_j)
                p_k = np.sum(pa_k)

                bic_cand = bic_curr.copy()
                #2pj+3 -> 2pj+1
                bic_cand[j] = -2 * out_j.fun + (2 * p_j + 3) * np.log(n)
                bic_cand[k] = -2 * out_k.fun + (2 * p_k + 3) * np.log(n)
                print("cand bic: ",np.round(np.sum(bic_cand),4))
                print("iter bic: ",np.round(np.sum(bic_iter),4))

                if np.isfinite(np.sum(bic_cand)) and np.sum(bic_cand) < np.sum(bic_iter) - tol:

                    IMPROV = True
                    bic_iter = bic_cand
                    est_iter = est_curr.copy()
                    est_iter["E"] = E_cand.copy()
                    est_iter["alpha"][j, k] = 0
                    est_iter["beta"][j, k] = 0

                    if p_j > 0:
                        est_iter["alpha"][j, pa_j] = out_j.x[:p_j]
                        est_iter["beta"][j, pa_j] = out_j.x[(p_j + 1) : (2 * p_j + 1)]
                    est_iter["delta"][j] = out_j.x[p_j]
                    est_iter["gamma"][j] = out_j.x[2 * p_j + 1]
                    est_iter["lambda"][j] = np.exp(out_j.x[2 * p_j + 2])

                    est_iter["alpha"][k, pa_k] = out_k.x[:p_k]
                    est_iter["beta"][k, pa_k] = out_k.x[(p_k + 1) : (2 * p_k + 1)]   
                    est_iter["delta"][k] = out_k.x[p_k]
                    est_iter["gamma"][k] = out_k.x[2 * p_k + 1]
                    est_iter["lambda"][k] = np.exp(out_k.x[2 * p_k + 2])

        print("iter =", Iter+1, "; BIC =", np.round(np.sum(bic_iter), 4), "\n")
        print('E: \n', est_iter["E"])

        if (not IMPROV):
            print('no improve')
            break
        
        bic_curr = bic_iter.copy()
        est_curr = est_iter.copy()
    
    out = {"est": est_curr, "bic": bic_curr, "iter": Iter}
    
    return out

In [570]:
simulation = {'1':0, '2':0,'3':0,'4':0,'5':0}

for i in range(6):
    file = 'syn_data.csv'
    dat = pd.read_csv(file,sep=',')
    dat = np.array(dat)
    
    out = hc_linear_zihp(dat, starting_dag = None, maxiter=500, tol=1e-12, verbose=False)
    simulation[str(i)] = out['est']['E']

/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/4258727766.py:14: RuntimeWarning: overflow encountered in add
  series = temp + fac
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/4258727766.py:11: RuntimeWarning: overflow encountered in multiply
  fac *= ((c/d) * (p/(e+1)))
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/4258727766.py:34: RuntimeWarning: overflow encountered in multiply
  fac *= (digamma(c+1)/digamma(c)) * (z/c)
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/4258727766.py:97: RuntimeWarning: invalid value encountered in true_divide
  p2 = -np.sum(X11 * (Mu[Y1] / Lambda * eval2_F11[Y1] / eval1_F11[Y1] -y[Y1].reshape(-1,1)), axis = 0)-np.sum(X10 * (Mu[Y0] * (1-Pi[Y0]) / Lambda * eval2_F11[Y0] / np.square(eval1_F11[Y0])/dZIHP0), axis = 0)
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/4258727766.py:103: RuntimeWarning: invalid value encountered in true_divide
  p3 = (-np.sum(np.multipl

/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/4258727766.py:103: RuntimeWarning: invalid value encountered in true_divide
  p3 = (-np.sum(np.multiply((1-Pi[Y0]), eval_gradF11b[Y0]/np.square(eval1_F11[Y0])/dZIHP0), axis = 0)-np.sum((eval_gradF11b[Y1]/eval1_F11[Y1] + digamma(Lambda + y[Y1].reshape(-1,1)) - digamma(Lambda)), axis = 0)) * Lambda


Add Edge from 0 to 1
cand bic:  1378.07
iter bic:  1375.3345
Add Edge from 0 to 2


/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/4258727766.py:11: RuntimeWarning: overflow encountered in multiply
  fac *= ((c/d) * (p/(e+1)))
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/4258727766.py:34: RuntimeWarning: overflow encountered in multiply
  fac *= (digamma(c+1)/digamma(c)) * (z/c)
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/4258727766.py:97: RuntimeWarning: invalid value encountered in true_divide
  p2 = -np.sum(X11 * (Mu[Y1] / Lambda * eval2_F11[Y1] / eval1_F11[Y1] -y[Y1].reshape(-1,1)), axis = 0)-np.sum(X10 * (Mu[Y0] * (1-Pi[Y0]) / Lambda * eval2_F11[Y0] / np.square(eval1_F11[Y0])/dZIHP0), axis = 0)
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/4258727766.py:103: RuntimeWarning: invalid value encountered in true_divide
  p3 = (-np.sum(np.multiply((1-Pi[Y0]), eval_gradF11b[Y0]/np.square(eval1_F11[Y0])/dZIHP0), axis = 0)-np.sum((eval_gradF11b[Y1]/eval1_F11[Y1] + digamma(Lambda + y[Y1].reshape(

cand bic:  1475.4944
iter bic:  1375.3345
Add Edge from 0 to 3


...

cand bic:  1516.8693
iter bic:  1372.575
Add Edge from 1 to 3


/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/4258727766.py:14: RuntimeWarning: overflow encountered in add
  series = temp + fac
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/4258727766.py:34: RuntimeWarning: overflow encountered in multiply
  fac *= (digamma(c+1)/digamma(c)) * (z/c)
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/4258727766.py:37: RuntimeWarning: overflow encountered in add
  series = temp + fac
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/4258727766.py:97: RuntimeWarning: invalid value encountered in true_divide
  p2 = -np.sum(X11 * (Mu[Y1] / Lambda * eval2_F11[Y1] / eval1_F11[Y1] -y[Y1].reshape(-1,1)), axis = 0)-np.sum(X10 * (Mu[Y0] * (1-Pi[Y0]) / Lambda * eval2_F11[Y0] / np.square(eval1_F11[Y0])/dZIHP0), axis = 0)
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/4258727766.py:103: RuntimeWarning: invalid value encountered in true_divide
  p3 = (-np.sum(np.multiply((1-Pi[Y0])

cand bic:  2107.752
iter bic:  1372.575
Add Edge from 1 to 4


/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/4258727766.py:11: RuntimeWarning: overflow encountered in multiply
  fac *= ((c/d) * (p/(e+1)))
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/4258727766.py:34: RuntimeWarning: overflow encountered in multiply
  fac *= (digamma(c+1)/digamma(c)) * (z/c)
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/4258727766.py:97: RuntimeWarning: invalid value encountered in true_divide
  p2 = -np.sum(X11 * (Mu[Y1] / Lambda * eval2_F11[Y1] / eval1_F11[Y1] -y[Y1].reshape(-1,1)), axis = 0)-np.sum(X10 * (Mu[Y0] * (1-Pi[Y0]) / Lambda * eval2_F11[Y0] / np.square(eval1_F11[Y0])/dZIHP0), axis = 0)
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/4258727766.py:103: RuntimeWarning: invalid value encountered in true_divide
  p3 = (-np.sum(np.multiply((1-Pi[Y0]), eval_gradF11b[Y0]/np.square(eval1_F11[Y0])/dZIHP0), axis = 0)-np.sum((eval_gradF11b[Y1]/eval1_F11[Y1] + digamma(Lambda + y[Y1].reshape(

KeyboardInterrupt: 

In [317]:
TPR_list = []
FDR_list = []
for E in simulation.values():
    TPR_list.append(acc(E_true, E, 'TPR'))
    FDR_list.append(acc(E_true, E, 'FDR'))

In [326]:
data = [['TPR', np.round(np.mean(TPR_list),3)], ['FDR', np.round(np.mean(FDR_list),3)]]
df = pd.DataFrame(data, columns=["Measure"," Sample Size (n=50)"])
df = df.rename(index={0: "HC0", 1: "HC0"})
df

,Measure,Sample Size (n=50)
HC0,TPR,0.450
HC0,FDR,0.267


In [8]:
E_true

array([[0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0]])

In [ ]:
true = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, j], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)
         , x0=pars, jac=lambda z: gradZIHP_cpp(z, dat[:, j], dat[:, pa_j]), method='BFGS')

In [701]:
n, p = dat.shape

E_true  = np.zeros((p, p), dtype=int)

for j in range(1, p):
    E_true[j, j - 1] = 1

alpha_true = np.zeros((p, p))
beta_true = np.zeros((p, p))
delta_true = np.random.uniform(-1.5, -1, size=p)
gamma_true = np.random.uniform(1, 1.5, size=p)
lambda_true = np.exp(np.random.uniform(-2, 2, size=p))

alpha_true[1,0] = 0.898
alpha_true[2,1] = 1.058
alpha_true[3,2] = 1.359
alpha_true[4,3] = 1.862

beta_true[1,0] = -1.694
beta_true[2,1] = -0.652
beta_true[3,2] = -0.583
beta_true[4,3] = -1.009

delta_true[0] = -1.185
delta_true[1] = -1.469
delta_true[2] = -1.397
delta_true[3] = -1.411
delta_true[4] = -1.156

gamma_true[0] = 1.192
gamma_true[1] = 1.384
gamma_true[2] = 1.249
gamma_true[3] = 1.359
gamma_true[4] = 1.496

lambda_true[0] = 0.619
lambda_true[1] = 3.033
lambda_true[2] = 5.691
lambda_true[3] = 0.316
lambda_true[4] = 1.834

starting_dag = E_true

bic_curr = np.empty(p)
est_curr = {
    "E": starting_dag,
    "alpha": alpha_true,
    "beta": beta_true,
    "delta": delta_true,
    "gamma": gamma_true,
    "lambda": lambda_true,
}



In [702]:
est_curr

{'E': array([[0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0]]),
 'alpha': array([[0.   , 0.   , 0.   , 0.   , 0.   ],
        [0.898, 0.   , 0.   , 0.   , 0.   ],
        [0.   , 1.058, 0.   , 0.   , 0.   ],
        [0.   , 0.   , 1.359, 0.   , 0.   ],
        [0.   , 0.   , 0.   , 1.862, 0.   ]]),
 'beta': array([[ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [-1.694,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 0.   , -0.652,  0.   ,  0.   ,  0.   ],
        [ 0.   ,  0.   , -0.583,  0.   ,  0.   ],
        [ 0.   ,  0.   ,  0.   , -1.009,  0.   ]]),
 'delta': array([-1.185, -1.469, -1.397, -1.411, -1.156]),
 'gamma': array([1.192, 1.384, 1.249, 1.359, 1.496]),
 'lambda': array([0.619, 3.033, 5.691, 0.316, 1.834])}

In [573]:
pa_j = (est_curr['E'][j, :] == 1)

if np.array(est_curr["alpha"][j, pa_j]).size == 0 and np.array(est_curr["beta"][j, pa_j]).size == 0:
    start_j = [est_curr["delta"][j], est_curr["gamma"][j],
                np.log(est_curr["lambda"][j])]
else:
    start_j = [(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                np.log(est_curr["lambda"][j])]

In [803]:
def f11_cpp(p, a, b, Iter, tol):
    
    c = np.copy(a)
    d = np.copy(b)
    
    fac = 1
    temp = copy.copy(fac)
    series = 0
    
    for e in range(Iter):
        fac *= ((c/d) * (p/(e+1)))
        
        if np.isnan(fac): fac = 0
        series = temp + fac
        
        if (not np.isfinite(series) or np.abs(series - temp) < tol): 
            return series
        
        temp = series
        
        c += 1
        d += 1
    return series


def f11_for_each(z, a, b, Iter, tol):
    
    '''
    
    z: vector Mu_vector is 2D vector
    x: 1
    y: lambda
    
    '''
    
    num = len(z)
    val = np.zeros((num, 1))
    
    for i in range(num):
        val[i, 0] = f11_cpp(z[i, 0], a, b, Iter, tol)
    
    return np.array(val)

def gradF11b_cpp(z, b, Iter, tol):
    
    e = np.copy(b)
    
    con = digamma(e) * f11_cpp(z, 1, e, Iter, tol)
    fac = digamma(e)
    temp = copy.copy(fac)
    series = 0
    for i in range(Iter):
        fac *= (digamma(e+1)/digamma(e)) * (z/e)
        
        if np.isnan(fac): fac = 0
        series = temp + fac
        
        if (not np.isfinite(series)) or (np.abs(series - temp) < tol):
            return (con - series)
        
        temp = series
        e += 1
        
    return (con-series)

def gradF11b_for_each(z, b, Iter, tol):
    
    '''
    
    z: vector Mu_vector is 2D vector
    x: 1
    y: lambda
    
    '''
    
    num = len(z)
    val = np.empty((num, 1))

    for i in range(num):
        val[i, 0] = gradF11b_cpp(z[i, 0], b, Iter, tol)
    return val

def dZIHP_cpp(z, y, x, lower, upper):

    z = z.reshape(1,-1)

    pn, pp = x.shape
    x1 = np.concatenate([x, np.ones((n,1))], axis = 1)

    Pi = np.exp(x1 @ z[:,0:pp+1].T)
    Pi = Pi/(1+Pi)
    Pi[~np.isfinite(Pi)] = 1
    
    Mu = np.exp(x1 @ z[:,pp+1:2*pp+2].T)
    Lambda = np.copy(np.exp(z[:,-1]))
    
    Y0 = np.where(y<=1e-8)
    Y1 = np.where(y>1e-8)
    eval_F11 = f11_for_each(Mu, 1, Lambda, 10000, 1e-8)
    
    #return Y0, Y1, eval_F11, Pi, Mu
    # keep working on likelihood function
    llik = np.sum(np.log(Pi[Y0] + (1-Pi[Y0])/eval_F11[Y0]))+np.sum(np.log(1-Pi[Y1])-log_ascfacto(Lambda, y[Y1])-np.log(eval_F11[Y1]) + (y[Y1].reshape(-1,1) % np.log(Mu[Y1])))

    if ((not np.isfinite(llik)) and (llik<0)):
        return lower
    elif ((not np.isfinite(llik)) and (llik>0)):
        return upper
    else:
        return llik
    
def gradZIHP_cpp(z, y, x):
    
    z = z.reshape(1,-1)

    pn, pp = x.shape
    x1 = np.concatenate([x, np.ones((pn,1))], axis = 1)

    Pi = np.exp(x1 @ z[:,0:pp+1].T)
    Pi = Pi/(1+Pi)
    Pi[~np.isfinite(Pi)] = 1

    Mu = np.exp(x1 @ z[:,pp+1:2*pp+2].T)
    Lambda = np.copy(np.exp(z[:,-1]))
    
    #print('Lambda1: ', Lambda)
    Y0 = np.where(y<1e-8)
    Y1 = np.where(y>=1e-8)
    X10 = x1[Y0]
    X11 = x1[Y1]
    
    eval1_F11 = f11_for_each(Mu, 1, Lambda, 10000, 1e-8)
    eval2_F11 = f11_for_each(Mu, 2, Lambda+1, 10000, 1e-8)
    eval_gradF11b = gradF11b_for_each(Mu, Lambda, 10000, 1e-8)
    
    #print('Lambda2: ', Lambda)
    dZIHP0 = Pi[Y0] + (1-Pi[Y0])/eval1_F11[Y0]
    
    grad = np.zeros(2 * pp + 3)
    p1 = np.sum(np.multiply(X10, np.multiply((1 - Pi[Y0]),(Pi[Y0] - Pi[Y0]/ eval1_F11[Y0])/dZIHP0)), axis = 0)-np.sum(np.multiply(X11, Pi[Y1]), axis = 0)

    # C++ code for p2
    #p2 =-arma::sum(X10.each_col() % (Mu.elem(Y0) % (1 - Pi.elem(Y0)) / Lambda % eval2_F11.elem(Y0) / arma::square(eval1_F11.elem(Y0)) / dZIHP0), 0) -
        #arma::sum(X11.each_col() % (Mu.elem(Y1) / Lambda % eval2_F11.elem(Y1) / eval1_F11.elem(Y1) - y.elem(Y1)), 0);
    # rewrite 11/27
    p2 = -np.sum(X11 * (Mu[Y1] / Lambda * eval2_F11[Y1] / eval1_F11[Y1] -y[Y1].reshape(-1,1)), axis = 0)-np.sum(X10 * (Mu[Y0] * (1-Pi[Y0]) / Lambda * eval2_F11[Y0] / np.square(eval1_F11[Y0])/dZIHP0), axis = 0)

    # C++ code for p3  
    # -arma::accu((1 - Pi.elem(Y0)) % eval_gradF11b.elem(Y0) / arma::square(eval1_F11.elem(Y0)) / dZIHP0) - 
    # arma::accu(eval_gradF11b.elem(Y1) / eval1_F11.elem(Y1) + digamma_arma(Lambda + y.elem(Y1)) - R::digamma(Lambda))) * Lambda;
    #p3 = (-np.sum((1-Pi[Y0]) % eval_gradF11b[Y0]/np.square(eval1_F11[Y0])/dZIHP0), axis = 0) -np.sum((eval_gradF11b[Y1]/eval1_F11[Y1] + digamma(Lambda + y[Y1].reshape(-1,1)) - digamma(Lambda)), axis = 0)) * Lambda
    p3 = (-np.sum(np.multiply((1-Pi[Y0]), eval_gradF11b[Y0]/np.square(eval1_F11[Y0])/dZIHP0), axis = 0)-np.sum((eval_gradF11b[Y1]/eval1_F11[Y1] + digamma(Lambda + y[Y1].reshape(-1,1)) - digamma(Lambda)), axis = 0)) * Lambda
    grad[0:pp+1] = p1
    grad[pp+1: 2*pp+2] = p2
    grad[2*pp+2] = p3

    return grad

In [716]:
start_j

array([-1.185     ,  1.192     , -0.47965001])

In [714]:
j=0
pa_j = (est_curr['E'][j, :] == 1)

if np.array(est_curr["alpha"][j, pa_j]).size == 0 and np.array(est_curr["beta"][j, pa_j]).size == 0:
    start_j = [est_curr["delta"][j], est_curr["gamma"][j],
                np.log(est_curr["lambda"][j])]
else:
    start_j = [(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                np.log(est_curr["lambda"][j])]

start_j = initialpars(start_j)
pars = np.array(start_j)
out_j = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, j], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)
                 , x0=pars, jac=lambda z: gradZIHP_cpp(z, dat[:, j], dat[:, pa_j]), method='BFGS')

/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/3410521011.py:103: RuntimeWarning: divide by zero encountered in log
  llik = np.sum(np.log(Pi[Y0] + (1-Pi[Y0])/eval_F11[Y0]))+np.sum(np.log(1-Pi[Y1])-log_ascfacto(Lambda, y[Y1])-np.log(eval_F11[Y1]) + (y[Y1].reshape(-1,1) * np.log(Mu[Y1])))
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/3410521011.py:95: RuntimeWarning: overflow encountered in exp
  Lambda = np.copy(np.exp(z[:,-1]))
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/3410521011.py:124: RuntimeWarning: overflow encountered in exp
  Lambda = np.copy(np.exp(z[:,-1]))
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/3410521011.py:53: RuntimeWarning: invalid value encountered in true_divide
  fac *= (digamma(e+1)/digamma(e)) * (z/e)
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/3410521011.py:59: RuntimeWarning: invalid value encountered in subtract
  return (con - series)
/var/folders/2_/x3tlt3

In [676]:
bic = 0
for j in range(p):
    print('Parameter: ',j+1)
    pa_j = (est_curr['E'][j, :] == 1)

    if np.array(est_curr["alpha"][j, pa_j]).size == 0 and np.array(est_curr["beta"][j, pa_j]).size == 0:
        start_j = [est_curr["delta"][j], est_curr["gamma"][j],
                    np.log(est_curr["lambda"][j])]
    else:
        start_j = [(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                    np.log(est_curr["lambda"][j])]

    start_j = initialpars(start_j)
    pars = np.array(start_j)
    out_j = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, j], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)
                     , x0=pars, jac=lambda z: gradZIHP_cpp(z, dat[:, j], dat[:, pa_j]), method='BFGS')
    print('loglikelihood: ',out_j.fun)
    print('parameter: ',out_j.x)
    bic += out_j.fun

Parameter:  1


/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/3410521011.py:146: RuntimeWarning: overflow encountered in square
  p2 = -np.sum(X11 * (Mu[Y1] / Lambda * eval2_F11[Y1] / eval1_F11[Y1] -y[Y1].reshape(-1,1)), axis = 0)-np.sum(X10 * (Mu[Y0] * (1-Pi[Y0]) / Lambda * eval2_F11[Y0] / np.square(eval1_F11[Y0])/dZIHP0), axis = 0)
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/3410521011.py:152: RuntimeWarning: overflow encountered in square
  p3 = (-np.sum(np.multiply((1-Pi[Y0]), eval_gradF11b[Y0]/np.square(eval1_F11[Y0])/dZIHP0), axis = 0)-np.sum((eval_gradF11b[Y1]/eval1_F11[Y1] + digamma(Lambda + y[Y1].reshape(-1,1)) - digamma(Lambda)), axis = 0)) * Lambda
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/3410521011.py:11: RuntimeWarning: overflow encountered in multiply
  fac *= ((c/d) * (p/(e+1)))
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/3410521011.py:53: RuntimeWarning: overflow encountered in multiply
  fac *= (digam

loglikelihood:  -103.97479780738023
parameter:  [-1.33667755  1.31168006 -1.31063797]
Parameter:  2


/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/3410521011.py:11: RuntimeWarning: overflow encountered in multiply
  fac *= ((c/d) * (p/(e+1)))
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/3410521011.py:14: RuntimeWarning: overflow encountered in add
  series = temp + fac
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/3410521011.py:53: RuntimeWarning: overflow encountered in multiply
  fac *= (digamma(e+1)/digamma(e)) * (z/e)
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/3410521011.py:146: RuntimeWarning: overflow encountered in square
  p2 = -np.sum(X11 * (Mu[Y1] / Lambda * eval2_F11[Y1] / eval1_F11[Y1] -y[Y1].reshape(-1,1)), axis = 0)-np.sum(X10 * (Mu[Y0] * (1-Pi[Y0]) / Lambda * eval2_F11[Y0] / np.square(eval1_F11[Y0])/dZIHP0), axis = 0)
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/3410521011.py:146: RuntimeWarning: invalid value encountered in true_divide
  p2 = -np.sum(X11 * (Mu[Y1] / Lamb

loglikelihood:  -28.61184088355779
parameter:  [ 1.1748682  -1.23647095 -0.64493072  1.00791062 -1.451457  ]
Parameter:  3


...

loglikelihood:  -42.71798166918464
parameter:  [ 1.36758442 -1.05702895 -1.13948077  1.46471862  1.73038185]
Parameter:  4


/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/3410521011.py:103: RuntimeWarning: divide by zero encountered in log
  llik = np.sum(np.log(Pi[Y0] + (1-Pi[Y0])/eval_F11[Y0]))+np.sum(np.log(1-Pi[Y1])-log_ascfacto(Lambda, y[Y1])-np.log(eval_F11[Y1]) + (y[Y1].reshape(-1,1) * np.log(Mu[Y1])))
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/3410521011.py:90: RuntimeWarning: overflow encountered in exp
  Pi = np.exp(x1 @ z[:,0:pp+1].T)
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/3410521011.py:91: RuntimeWarning: invalid value encountered in true_divide
  Pi = Pi/(1+Pi)
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/3410521011.py:95: RuntimeWarning: overflow encountered in exp
  Lambda = np.copy(np.exp(z[:,-1]))
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/3410521011.py:119: RuntimeWarning: overflow encountered in exp
  Pi = np.exp(x1 @ z[:,0:pp+1].T)
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/i

loglikelihood:  nan
parameter:  [-100476.31330352  -66677.8926479   466300.01330953 1800108.96104332
 -799156.3714848 ]
Parameter:  5


/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/3410521011.py:146: RuntimeWarning: overflow encountered in square
  p2 = -np.sum(X11 * (Mu[Y1] / Lambda * eval2_F11[Y1] / eval1_F11[Y1] -y[Y1].reshape(-1,1)), axis = 0)-np.sum(X10 * (Mu[Y0] * (1-Pi[Y0]) / Lambda * eval2_F11[Y0] / np.square(eval1_F11[Y0])/dZIHP0), axis = 0)
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/3410521011.py:152: RuntimeWarning: overflow encountered in square
  p3 = (-np.sum(np.multiply((1-Pi[Y0]), eval_gradF11b[Y0]/np.square(eval1_F11[Y0])/dZIHP0), axis = 0)-np.sum((eval_gradF11b[Y1]/eval1_F11[Y1] + digamma(Lambda + y[Y1].reshape(-1,1)) - digamma(Lambda)), axis = 0)) * Lambda
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/3410521011.py:11: RuntimeWarning: overflow encountered in multiply
  fac *= ((c/d) * (p/(e+1)))
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_56577/3410521011.py:53: RuntimeWarning: overflow encountered in multiply
  fac *= (digam

loglikelihood:  -44.273976437945315
parameter:  [ 0.85554047 -1.04573242 -1.07428263  1.49866143 -1.73599931]


## gradZIHP

In [828]:
def gradZIHP_cpp(z, y, x):
    
    z = z.reshape(1,-1)

    pn, pp = x.shape
    x1 = np.concatenate([x, np.ones((pn,1))], axis = 1)

    Pi = np.exp(x1 @ z[:,0:pp+1].T)
    Pi = Pi/(1+Pi)
    Pi[~np.isfinite(Pi)] = 1

    Mu = np.exp(x1 @ z[:,pp+1:2*pp+2].T)
    Lambda = np.copy(np.exp(z[:,-1]))
    
    #print('Lambda1: ', Lambda)
    Y0 = np.where(y<1e-8)
    Y1 = np.where(y>=1e-8)
    X10 = x1[Y0]
    X11 = x1[Y1]
    
    eval1_F11 = f11_for_each(Mu, 1, Lambda, 10000, 1e-8)
    eval2_F11 = f11_for_each(Mu, 2, Lambda+1, 10000, 1e-8)
    eval_gradF11b = gradF11b_for_each(Mu, Lambda, 10000, 1e-8)
    
    #print('Lambda2: ', Lambda)
    dZIHP0 = Pi[Y0] + (1-Pi[Y0])/eval1_F11[Y0]
    
    grad = np.zeros(2 * pp + 3)
    p1 = np.sum(np.multiply(X10, np.multiply((1 - Pi[Y0]),(Pi[Y0] - Pi[Y0]/ eval1_F11[Y0])/dZIHP0)), axis = 0)-np.sum(np.multiply(X11, Pi[Y1]), axis = 0)

    # C++ code for p2
    #p2 =-arma::sum(X10.each_col() % (Mu.elem(Y0) % (1 - Pi.elem(Y0)) / Lambda % eval2_F11.elem(Y0) / arma::square(eval1_F11.elem(Y0)) / dZIHP0), 0) -
        #arma::sum(X11.each_col() % (Mu.elem(Y1) / Lambda % eval2_F11.elem(Y1) / eval1_F11.elem(Y1) - y.elem(Y1)), 0);
    # rewrite 11/27
    p2 = -np.sum(X11 * (Mu[Y1] / Lambda * eval2_F11[Y1] / eval1_F11[Y1] -y[Y1].reshape(-1,1)), axis = 0)-np.sum(X10 * (Mu[Y0] * (1-Pi[Y0]) / Lambda * eval2_F11[Y0] / np.square(eval1_F11[Y0])/dZIHP0), axis = 0)

    # C++ code for p3  
    # -arma::accu((1 - Pi.elem(Y0)) % eval_gradF11b.elem(Y0) / arma::square(eval1_F11.elem(Y0)) / dZIHP0) - 
    # arma::accu(eval_gradF11b.elem(Y1) / eval1_F11.elem(Y1) + digamma_arma(Lambda + y.elem(Y1)) - R::digamma(Lambda))) * Lambda;
    #p3 = (-np.sum((1-Pi[Y0]) % eval_gradF11b[Y0]/np.square(eval1_F11[Y0])/dZIHP0), axis = 0) -np.sum((eval_gradF11b[Y1]/eval1_F11[Y1] + digamma(Lambda + y[Y1].reshape(-1,1)) - digamma(Lambda)), axis = 0)) * Lambda
    p3 = (-np.sum(np.multiply((1-Pi[Y0]), eval_gradF11b[Y0]/np.square(eval1_F11[Y0])/dZIHP0), axis = 0)-np.sum((eval_gradF11b[Y1]/eval1_F11[Y1] + digamma(Lambda + y[Y1].reshape(-1,1)) - digamma(Lambda)), axis = 0)) * Lambda
    grad[0:pp+1] = p1
    grad[pp+1: 2*pp+2] = p2
    grad[2*pp+2] = p3

    return grad

In [829]:
j=0
pa_j = (est_curr['E'][j, :] == 1)

if np.array(est_curr["alpha"][j, pa_j]).size == 0 and np.array(est_curr["beta"][j, pa_j]).size == 0:
    start_j = [est_curr["delta"][j], est_curr["gamma"][j],
                np.log(est_curr["lambda"][j])]
else:
    start_j = [(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                np.log(est_curr["lambda"][j])]

start_j = initialpars(start_j)
pars = np.array(start_j)

z = pars
y = dat[:, j]
x = dat[:, pa_j]
lower = -np.finfo(float).max
upper = np.finfo(float).max

z = z.reshape(1,-1)

pn, pp = x.shape
x1 = np.concatenate([x, np.ones((pn,1))], axis = 1)

Pi = np.exp(x1 @ z[:,0:pp+1].T)
Pi = Pi/(1+Pi)
Pi[~np.isfinite(Pi)] = 1

Mu = np.exp(x1 @ z[:,pp+1:2*pp+2].T)
Lambda = np.copy(np.exp(z[:,-1]))

Y0 = np.where(np.abs(y)<1e-8)
Y1 = np.where(np.abs(y)>=1e-8)
X10 = x1[Y0]
X11 = x1[Y1]

eval1_F11 = f11_for_each(Mu, 1, Lambda, 10000, 1e-8)
eval2_F11 = f11_for_each(Mu, 2, Lambda+1, 10000, 1e-8)
eval_gradF11b = gradF11b_for_each(Mu, Lambda, 10000, 1e-8)

dZIHP0 = Pi[Y0] + (1-Pi[Y0])/eval1_F11[Y0]
    
grad = np.zeros(2 * pp + 3)
# C++ code for p1
# p1 = np.sum(X10 % ((1 - Pi[Y0]) % ((Pi[Y0] - (Pi[Y0]/ eval1_F11[Y0])) / dZIHP0)), axis = 0) - np.sum(X11 % Pi[Y1], axis =0) 
# rewrite 11/27
#p1 = np.sum(X10 % ((1 - Pi[Y0]) % (Pi[Y0] - Pi[Y0]/ eval1_F11[Y0]) / dZIHP0), axis = 0) - np.sum(X11 % Pi[Y1], axis =0) 
p1 = np.sum(X10 * ((1-Pi[Y0]) * (Pi[Y0] - Pi[Y0]/eval1_F11[Y0])/dZIHP0), axis = 0)-np.sum(X11 * Pi[Y1], axis = 0)

# C++ code for p2
#p2 =-arma::sum(X10.each_col() % (Mu.elem(Y0) % (1 - Pi.elem(Y0)) / Lambda % eval2_F11.elem(Y0) / arma::square(eval1_F11.elem(Y0)) / dZIHP0), 0) -
    #arma::sum(X11.each_col() % (Mu.elem(Y1) / Lambda % eval2_F11.elem(Y1) / eval1_F11.elem(Y1) - y.elem(Y1)), 0);
# rewrite 11/27
p2 = -np.sum(X11 * (Mu[Y1] / Lambda * eval2_F11[Y1] / eval1_F11[Y1] -y[Y1].reshape(-1,1)), axis = 0)-np.sum(X10 * (Mu[Y0] * (1-Pi[Y0]) / Lambda * eval2_F11[Y0] / np.square(eval1_F11[Y0])/dZIHP0), axis = 0)

# C++ code for p3  
# -arma::accu((1 - Pi.elem(Y0)) % eval_gradF11b.elem(Y0) / arma::square(eval1_F11.elem(Y0)) / dZIHP0) - 
# arma::accu(eval_gradF11b.elem(Y1) / eval1_F11.elem(Y1) + digamma_arma(Lambda + y.elem(Y1)) - R::digamma(Lambda))) * Lambda;
#p3 = (-np.sum((1-Pi[Y0]) % eval_gradF11b[Y0]/np.square(eval1_F11[Y0])/dZIHP0), axis = 0) -np.sum((eval_gradF11b[Y1]/eval1_F11[Y1] + digamma(Lambda + y[Y1].reshape(-1,1)) - digamma(Lambda)), axis = 0)) * Lambda
p3 = (-np.sum(np.multiply((1-Pi[Y0]), eval_gradF11b[Y0]/np.square(eval1_F11[Y0])/dZIHP0), axis = 0)-np.sum((eval_gradF11b[Y1]/eval1_F11[Y1] + digamma(Lambda + y[Y1].reshape(-1,1)) - digamma(Lambda)), axis = 0)) * Lambda
grad[0:pp+1] = p1
grad[pp+1: 2*pp+2] = p2
grad[2*pp+2] = p3


In [831]:
gradZIHP_cpp(pars, dat[:, j], dat[:, pa_j])

array([-0.3138029 ,  4.37491378, -1.01841497])

## dZIHP_cpp

In [816]:
def dZIHP_cpp(z, y, x, lower, upper):

    z = z.reshape(1,-1)

    pn, pp = x.shape
    x1 = np.concatenate([x, np.ones((pn,1))], axis = 1)

    Pi = np.exp(x1 @ z[:,0:pp+1].T)
    Pi = Pi/(1+Pi)
    Pi[~np.isfinite(Pi)] = 1

    Mu = np.exp(x1 @ z[:,pp+1:2*pp+2].T)
    Lambda = np.copy(np.exp(z[:,-1]))

    Y0 = np.where(np.abs(y)<1e-8)
    Y1 = np.where(np.abs(y)>=1e-8)
    eval1_F11 = f11_for_each(Mu, 1, Lambda, 10000, 1e-8)

    llik = np.sum(np.log(Pi[Y0]+(1-Pi[Y0])/eval1_F11[Y0])) + np.sum(np.log(1-Pi[Y1])-log_ascfacto(Lambda, y[Y1])-np.log(eval1_F11[Y1]) + y[Y1].reshape(-1,1) * np.log(Mu[Y1]))
    
    if ((not np.isfinite(llik)) and (llik<0)):
        return lower
    elif ((not np.isfinite(llik)) and (llik>0)):
        return upper
    else:
        return llik

In [808]:
j=0
pa_j = (est_curr['E'][j, :] == 1)

if np.array(est_curr["alpha"][j, pa_j]).size == 0 and np.array(est_curr["beta"][j, pa_j]).size == 0:
    start_j = [est_curr["delta"][j], est_curr["gamma"][j],
                np.log(est_curr["lambda"][j])]
else:
    start_j = [(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                np.log(est_curr["lambda"][j])]

start_j = initialpars(start_j)
pars = np.array(start_j)

z = pars
y = dat[:, j]
x = dat[:, pa_j]
lower = -np.finfo(float).max
upper = np.finfo(float).max

z = z.reshape(1,-1)

pn, pp = x.shape
x1 = np.concatenate([x, np.ones((pn,1))], axis = 1)

Pi = np.exp(x1 @ z[:,0:pp+1].T)
Pi = Pi/(1+Pi)
Pi[~np.isfinite(Pi)] = 1

Mu = np.exp(x1 @ z[:,pp+1:2*pp+2].T)
Lambda = np.copy(np.exp(z[:,-1]))

Y0 = np.where(np.abs(y)<1e-8)
Y1 = np.where(np.abs(y)>=1e-8)
eval1_F11 = f11_for_each(Mu, 1, Lambda, 10000, 1e-8)

# arma::accu(arma::log(Pi.elem(Y0) + (1 - Pi.elem(Y0)) / eval_F11.elem(Y0))) + 
# arma::accu(arma::log(1 - Pi.elem(Y1)) - log_ascfacto(Lambda, y.elem(Y1)) - arma::log(eval_F11.elem(Y1)) + y.elem(Y1) % arma::log(Mu.elem(Y1)));
  
llik = np.sum(np.log(Pi[Y0]+(1-Pi[Y0])/eval1_F11[Y0])) + np.sum(np.log(1-Pi[Y1])-log_ascfacto(Lambda, y[Y1])-np.log(eval1_F11[Y1]) + y[Y1].reshape(-1,1) * np.log(Mu[Y1]))

if ((not np.isfinite(llik)) and (llik<0)):
    print(lower)
elif ((not np.isfinite(llik)) and (llik>0)):
    print(upper)
else:
    print(llik)

-102.00445621187393


In [826]:
j=4
pa_j = (est_curr['E'][j, :] == 1)

if np.array(est_curr["alpha"][j, pa_j]).size == 0 and np.array(est_curr["beta"][j, pa_j]).size == 0:
    start_j = [est_curr["delta"][j], est_curr["gamma"][j],
                np.log(est_curr["lambda"][j])]
else:
    start_j = [(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                np.log(est_curr["lambda"][j])]

start_j = initialpars(start_j)
pars = np.array(start_j)

z = pars
y = dat[:, j]
x = dat[:, pa_j]

In [849]:
bic = 0
for j in range(p):
    pa_j = (est_curr['E'][j, :] == 1)
    n, p = dat.shape
    if np.array(est_curr["alpha"][j, pa_j]).size == 0 and np.array(est_curr["beta"][j, pa_j]).size == 0:
        start_j = [est_curr["delta"][j], est_curr["gamma"][j],
                    np.log(est_curr["lambda"][j])]
    else:
        start_j = [(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                    np.log(est_curr["lambda"][j])]

    start_j = initialpars(start_j)
    pars = np.array(start_j)
    
    p_j = sum(pa_j)
    print('number of p_j: ', p_j)
    bic += -2*(dZIHP_cpp(pars, dat[:, j], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)) + (2*p_j+3)*np.log(n)

number of p_j:  0
number of p_j:  1
number of p_j:  1
number of p_j:  1
number of p_j:  1


In [850]:
bic

689.9926499716138

In [835]:
p_j = sum(pa_j)

In [837]:
p_j

0